In [1]:
from PyPDF2 import PdfReader
import chromadb
PATH = "pdf/Learning LangChain-O'Reilly Media (2024).pdf"


reader = PdfReader(PATH)
        
text_load = ''.join(page.extract_text() for page in reader.pages)

client = chromadb.PersistentClient(path="chromadb")
collection = client.get_or_create_collection(name="vector-langchain") 


In [2]:
text_split = []
metadata=[]
chunk_size = 512
chunk_overlap = 8
        
for i in range(0, len(text_load), chunk_size):
    start = i
    end = i + chunk_size
    if start != 0:
        start = start - chunk_overlap
        end =  end + chunk_overlap
    text_split.append(text_load[start:end])
    metadata.append({"name": "book-langchain", "partition": f"{i}"})
    

In [3]:
import ollama

for i, d in enumerate(text_split):
        response = ollama.embeddings(model="mxbai-embed-large",
                                       prompt=d)
        
        collection.add(
                ids=[str(i)],
                embeddings=response['embedding'],
                documents=[d],
                metadatas=metadata[i]
    )

In [22]:
prompt = "what is python?"

response = ollama.embeddings(
    prompt=prompt,
    model="mxbai-embed-large"
    )

results = collection.query(
    query_embeddings=[response["embedding"]],
    n_results=3,
    where={"name": "book-langchain"}
    )
    
distance = results['distances'][0][0]
print(distance)

data = results['documents']
print(data)

233.73208618164062
[[' APIkey, and save it—you’ll need it soon.\nNOTE\nIn this book we’ll show code examples in both Python and JavaScript,\nLangChain oﬀers the same functionality in both languages, so just pick\none of them and follow the respective code snippets throughout the\nbook. The code examples for each language will be equivalent between\nthe two, so just pick whichever language you’re most comfortable with.\nFirst, some setup instructions for readers using Python:\n1. Ensure you have Python installed. See instructions here\nfor your operating', 'hain_text_splitters import (  \n    Language,  \n    RecursiveCharacterTextSplitter,  \n) And in JS:PYTHON_CODE = """  \ndef hello_world():  \n    print("Hello, World!")  \n# Call the function  \nhello_world()  \n""" \npython_splitter = RecursiveCharacterTextSplitter\n    language=Language.PYTHON, chunk_size=50, chun\n) \npython_docs = python_splitter.create_documents([P\n \nimport { RecursiveCharacterTextSplitter } from "@\nconst PY

In [23]:
output = ollama.generate(
        model="llama2",
        prompt=f"You are a chatbot assistant that responds in Portuguese. Using this data: {data}. Respond to this prompt: {prompt}. If the {distance} is more than 200, say you don't know the answer"
    )

print(output['response'])

Olá! Em Portuguese, "Python" é um linguagem de programação popular e amplamente utilizada para desenvolver aplicativos desktop, web e móveis. Ela é conhecida por sua sintaxe clara e fácil de aprender, além de sua vastidiade de bibliotecas e recursos disponíveis para ajudar no desenvolvimento de aplicativos.

Se você está procurando saber mais sobre a linguagem Python, pode consultar documentaçãooficial ou usar recursos on-line para aprender e entender melhor sua sintaxe e funcionalidades.
